In [1]:
# Example on how to use PopulationData class for preprocessing
import glob
import numpy as np
from joblib import Parallel, delayed
from tqdm import tqdm
from pathlib import Path
from typing import Dict, List
from ephysvibe.structures.neuron_data import NeuronData
from ephysvibe.structures.population_data import PopulationData
import pandas as pd

In [2]:
# paths
filepaths = {
    "lip": "D:/data/session_struct/lip/neurons/",
    "pfc": "",
    "v4": "",
}

In [3]:
path_list={}
for area in filepaths.keys():
    path = filepaths[area]
    neu_path = path + "*neu.h5"
    path_list[area] = glob.glob(neu_path)

### Create a population object: option 1

In [4]:
# Create a population by reading neurons and casting
## all attributes contaning nans must be at least float16 
## it is possible to replace nans by a specific number like for example -3 
attr_dtype = {'sp_samples':np.int8,'cluster_ch':np.float16,'cluster_number':np.float16,'block':np.float16,'trial_error':np.float16,'sample_id':np.float16,
              'test_stimuli':np.float16,'test_distractor':np.float16,'cluster_id':np.float16,'cluster_depth':np.float16,'code_samples':np.float16,
              'position':np.float16,'pos_code':np.float16,'code_numbers':np.float16}
replace_nan = {'sp_samples':-1}
popu=PopulationData.get_population(path_list['lip'][0:10], attr_dtype=attr_dtype,replace_nan=replace_nan,comment='population data example')

100%|██████████| 10/10 [00:00<00:00, 34.11it/s]


In [5]:
print(popu.population[0].sp_samples[:3])
print(popu.population[0].sp_samples.dtype)

[[ 0  0  0 ... -1 -1 -1]
 [ 0  0  0 ... -1 -1 -1]
 [ 0  0  0 ... -1 -1 -1]]
int8


In [6]:
# Save population
popu.to_python_hdf5('test_population.h5')

In [7]:
# Read population
popu=PopulationData.from_python_hdf5('test_population.h5')

In [8]:
df = popu.execute_function(NeuronData.get_sp_per_sec)

100%|██████████| 10/10 [00:00<00:00, 3297.67it/s]


### Create a population object: option 2

In [9]:
# Define a function that recives the path to a NeuronData file and returns a NeuronData object
def get_neu_align(path:Path, params:List, sp_sample:bool=False)->NeuronData:
    """ Read, align and add to the object the spiking activity.

    Args:
        path (Path): _description_
        params (List): list of dicts containing:
                        * "select_block"
                        * "inout"
                        * "event"
                        * "time_before"
                        * "st"
                        * "end"
                        * "sp"
                        * "dtype_sp"
                        * "mask"
                        * "dtype_mask"

        sp_sample (bool, optional): Whether to delete sp_sample data. Defaults to False.

    Returns:
        NeuronData object
    """
    neu = NeuronData.from_python_hdf5(path)
    for it in params:
        sp, mask = neu.align_on(
            select_block=it["select_block"],
            select_pos=it["inout"],
            event=it["event"],
            time_before=it["time_before"],
            error_type=0,
        )
        endt = it["time_before"] + it["end"]
        stt = it["time_before"] + it["st"]
        setattr(neu, it["sp"], np.array(sp[:, :endt], dtype=it["dtype_sp"]))
        setattr(neu, it["mask"], np.array(mask, dtype=it["dtype_mask"]))
        setattr(neu, "st_" + it["event"] + "_" + it["inout"], np.array(stt, dtype=int))
        setattr(
            neu,
            "time_before_" + it["event"] + "_" + it["inout"],
            np.array(it["time_before"], dtype=int),
        )

    if ~sp_sample:
        setattr(neu, "sp_samples", np.array([]))

    return neu

In [10]:
# Define parameters
params = [
    {
        "inout": "in",
        "sp": "sample_on_in",
        "mask": "mask_in",
        "event": "sample_on",
        "time_before": 300,
        "st": 0,
        "end": 1550,
        "select_block": 1,
        "win": 100,
        "dtype_sp": np.int8,
        "dtype_mask": bool,
    }
]
# # If you also want to aling data to out (or any other possibility) you add more dictionaries to the list with all the required information:
# params = [
#     {
#         "inout": "in",
#         "sp": "sample_on_in",
#         "mask": "mask_in",
#         "event": "sample_on",
#         "time_before": 300,
#         "st": 0,
#         "end": 1550,
#         "select_block": 1,
#         "win": 100,
#         "dtype_sp": np.int8,
#         "dtype_mask": bool,
#     },
#     {
#         "inout": "out",
#         "sp": "sample_on_out",
#         "mask": "mask_out",
#         "event": "sample_on",
#         "time_before": 300,
#         "st": 0,
#         "end": 1550,
#         "select_block": 1,
#         "win": 100,
#         "dtype_sp": np.int8,
#         "dtype_mask": bool,
#     },
# ]

In [11]:
population_list = Parallel(n_jobs=-1)(
    delayed(get_neu_align)(neu, params) for neu in tqdm(path_list['lip'][0:10])
)

100%|██████████| 10/10 [00:00<00:00, 2497.65it/s]


In [12]:
print(population_list[0].sample_on_in)
print(population_list[0].sample_on_in.shape)
print(population_list[0].sample_on_in.dtype)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
(265, 1850)
int8


In [13]:
population_list[0].__dict__

{'date_time': '2023-10-18_10-57-56',
 'subject': 'Riesling',
 'area': 'lip',
 'experiment': '1',
 'recording': '1',
 'sp_samples': array([], dtype=float64),
 'cluster_id': array(17, dtype=int64),
 'cluster_ch': array(21, dtype=int64),
 'cluster_group': 'mua',
 'cluster_number': array(3, dtype=int64),
 'cluster_array_pos': array(2, dtype=int64),
 'cluster_depth': array(1100.),
 'block': array([2., 2., 2., ..., 1., 1., 1.]),
 'trial_error': array([0., 0., 8., ..., 4., 4., 4.]),
 'code_samples': array([[1000., 1020., 1252., ...,   nan,   nan,   nan],
        [1000., 1008., 1017., ...,   nan,   nan,   nan],
        [1000., 1014., 1111., ...,   nan,   nan,   nan],
        ...,
        [1000., 1002., 1014., ...,   nan,   nan,   nan],
        [1000., 1003., 1015., ...,   nan,   nan,   nan],
        [1000., 1003., 1015., ...,   nan,   nan,   nan]]),
 'code_numbers': array([[ 9., 35.,  8., ..., nan, nan, nan],
        [ 9., 35.,  8., ..., nan, nan, nan],
        [ 9., 35.,  8., ..., nan, nan, n

In [14]:
comment = str(params)
population = PopulationData(population_list, comment=comment)

In [15]:
print(population.created)
print(population.comment)

19/08/2024 13:19:59
[{'inout': 'in', 'sp': 'sample_on_in', 'mask': 'mask_in', 'event': 'sample_on', 'time_before': 300, 'st': 0, 'end': 1550, 'select_block': 1, 'win': 100, 'dtype_sp': <class 'numpy.int8'>, 'dtype_mask': <class 'bool'>}]


### Perform a computation on each neuron

In [16]:
# Define a funtion which takes a neuron as imput
def get_fr(neu):
    res = {}
    res["nid"] = neu.get_neuron_id()
    start_sample_on = neu.time_before_sample_on_in
    meanfr = np.mean(neu.sample_on_in[:, start_sample_on:start_sample_on+150])
    res["mean_fr"] = meanfr*1000
    return res

In [17]:
# if ret_df=True returns a pandas DataFrame, else a list 
df_fr = population.execute_function(
    get_fr, n_jobs=-1, ret_df=True
)

100%|██████████| 10/10 [00:00<00:00, 1918.27it/s]


In [18]:
df_fr

,nid,mean_fr
0,mua3LIP2023-10-18_10-57-56Riesling,3.345912
1,mua4LIP2023-10-18_10-57-56Riesling,6.389937
2,mua5LIP2023-10-18_10-57-56Riesling,0.226415
3,mua6LIP2023-10-18_10-57-56Riesling,7.320755
4,mua7LIP2023-10-18_10-57-56Riesling,8.427673
5,good1LIP2023-10-18_10-57-56Riesling,8.050314
6,mua8LIP2023-10-18_10-57-56Riesling,1.534591
7,mua9LIP2023-10-18_10-57-56Riesling,0.528302
8,mua10LIP2023-10-18_10-57-56Riesling,2.691824
9,mua11LIP2023-10-18_10-57-56Riesling,3.220126


### Select a subpopulation

In [19]:
df_fr_5 = df_fr[df_fr['mean_fr']>5]
df_fr_5

,nid,mean_fr
1,mua4LIP2023-10-18_10-57-56Riesling,6.389937
3,mua6LIP2023-10-18_10-57-56Riesling,7.320755
4,mua7LIP2023-10-18_10-57-56Riesling,8.427673
5,good1LIP2023-10-18_10-57-56Riesling,8.050314


In [20]:
nid_list = df_fr_5['nid'].values
nid_list

array(['mua4LIP2023-10-18_10-57-56Riesling',
       'mua6LIP2023-10-18_10-57-56Riesling',
       'mua7LIP2023-10-18_10-57-56Riesling',
       'good1LIP2023-10-18_10-57-56Riesling'], dtype=object)

In [21]:
population.get_subpopulation(nid_list)

## Old stuff

In [6]:
df=pd.read_csv('C:/Users/camil/Documents/int/code/Users/losadac/preproc/testdf.csv')

In [9]:
mean_fr = (df['in_mean_fr_sample']+df['in_mean_fr_delay'])/2
sel_neu = df[mean_fr>1]['nid'].values

In [11]:
subpopu = popu.get_subpopulation(sel_neu)

In [12]:
subpopu.population[0].sp_test

AttributeError: 'list' object has no attribute 'population'

In [16]:
def get_sample_test(neu:NeuronData,params:Dict):
    time_before_sample=params['time_before_sample']
    time_before_sample=params['time_before_test']
    sp_sample, mask = neu.align_on(
        select_block=params['select_block'],
        select_pos=params['select_pos'],
        event="sample_on",
        time_before=params['time_before_sample'],
        error_type=0,
    )
    sp_test, mask = neu.align_on(
        select_block=params['select_block'],
        select_pos=params['select_pos'],
        event="test_on_1",
        time_before=params['time_before_test'],
        error_type=0,
    )
    idx_start_sample = time_before_sample + params['start_sample']
    idx_end_sample = time_before_sample + params['end_sample']
    idx_start_test = time_before_test + params['start_test']
    idx_end_test = time_before_test + params['end_test']

    new_values = {}
    new_values['sp_sample'] = np.array(sp_sample[:,idx_start_sample:idx_end_sample],dtype=np.int8)
    new_values['sp_test'] = np.array(sp_test[:,idx_start_test:idx_end_test],dtype=np.int8)
    
    for attr_name in [
        "block",
        "trial_error",
        "code_samples",
        "code_numbers",
        "position",
        "pos_code",
        "sample_id",
        "test_stimuli",
        "test_distractor",
    ]:
        new_values[attr_name] = getattr(neu, attr_name)[mask]
    neu.edit_attributes(new_values)

In [17]:
params = {
    # sample
    'time_before_sample': 500,
    'start_sample': -200,
    'end_sample': 450 + 400,
    # test
    'time_before_test': 500,
    'start_test': -400,
    'end_test': n_test * 450 + 200,
    'select_block':1,
    'select_pos':1,
    'event':"sample_on",
    'error_type':0,
}

In [13]:
subpopu = PopulationData(subpopu)

In [18]:
_=subpopu.execute_function(get_sample_test,params=params,n_jobs=1,ret_df=False)

100%|██████████| 4/4 [00:00<00:00, 12.10it/s]


In [19]:
subpopu.population[0].sp_sample.shape

(265, 1050)

In [7]:
df.head()

,nid,in_mean_fr_sample_NN,in_lat_max_fr_sample_NN,in_mean_max_fr_sample_NN,in_mean_fr_delay_NN,in_lat_max_fr_delay_NN,in_mean_max_fr_delay_NN,in_mean_fr_sample_N,in_lat_max_fr_sample_N,in_mean_max_fr_sample_N,...,out_mean_max_fr_sample_NN,out_mean_fr_delay_NN,out_lat_max_fr_delay_NN,out_mean_max_fr_delay_NN,out_mean_fr_sample_N,out_lat_max_fr_sample_N,out_mean_max_fr_sample_N,out_mean_fr_delay_N,out_lat_max_fr_delay_N,out_mean_max_fr_delay_N
0,good1LIP2023-10-18_10-57-56Riesling,6.492367,136,8.078053,1.131511,3,1.642847,5.684495,139,8.851400,...,7.281459,1.614321,4,1.671470,4.872106,70.0,6.508536,2.933936,129.0,3.224715
1,mua8LIP2023-10-18_10-57-56Riesling,1.017039,42,1.766470,2.805549,234,3.710537,1.833958,273,2.898548,...,2.087411,3.820307,321,4.744953,0.946610,349.0,1.440784,1.426995,335.0,2.209467
2,mua9LIP2023-10-18_10-57-56Riesling,0.491892,380,0.808868,1.021345,221,1.341812,0.692484,427,0.779571,...,1.006486,0.573862,176,0.592177,0.894014,103.0,1.303982,0.546876,0.0,0.750160
3,mua10LIP2023-10-18_10-57-56Riesling,1.563345,108,3.349809,0.494177,1,0.728646,1.973887,94,2.795180,...,0.817303,0.605666,37,0.906040,0.897311,394.0,1.148393,1.451206,379.0,2.199128
4,mua11LIP2023-10-18_10-57-56Riesling,2.032776,100,4.071189,1.638862,155,1.862226,1.083320,92,2.041631,...,1.619150,0.854565,392,1.025324,0.681207,384.0,1.096511,1.508641,271.0,1.896699


: 

In [ ]:
def get_matrix_position(neu_data,matrix_df,ch_start):
    ch_start = np.load(
        sessions_path
        + session
        + "/Record Node 102/experiment1/recording1/continuous/Acquisition_Board-100.Rhythm Data/KS"
        + area.upper()
        + "/channel_map.npy"
    )[0][0]
    cluster_ch = neu_data.cluster_ch - ch_start
    matrix_df = pd.read_csv(
        ch_pos_path + area + "_ch_pos.csv",
        header=0,
        index_col=0,
    )
    matrix = matrix_df.values
    matrix = matrix - matrix.min().min()
    row, col = np.where(cluster_ch == matrix)
    return row,col